In [1]:
from source.util import utils as u
from source import metrics, plots
from source import handshake, handshake2, scargc, hs
import sys
import time
import os
import psutil
import resource
import numpy as np
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

In [2]:
def doTrain(data, label, step, n_features, percent, epsilon, n_components):

    label_ = label

    size = len(data)
    mod = size % 10

    if mod != 0:
        data = data[0:(size - mod), :]
        label = label[0:(size - mod)]

    tam = int((size - mod)/10)
    # print(tam)
    # sys.exit(0)

    data = np.vsplit(data, 10)
    label = u.chunkify(label, 10)
    label_ = label_[0:(size-mod)]
    # print(len(data))
    # print(len(label))

    # percent = int(percent)
    # print(percent)
    # print(epsilon)
    acc_percent = []
    f1_percent = []
    mcc_percent = []
    predicted = []
    updt = []    
        
    pool_size = int( len(data[0])/100 * percent )

    k = epsilon
    score = 0.0
    f1 = 0.0
    mcc = 0.0
    
    if pool_size >= k:

        pred, updt = scargc.scargc_1NN(data, label_, data[0], label[0], data[1], label[1], pool_size, n_components, n_features, k)
        pred = np.array(pred, dtype=int)
        # print(label[i])
        pred = pred.flatten()
        predicted = pred
        acc_percent, f1_percent, mcc_percent = metrics.makeBatches(label[0], predicted, len(label[0]), step)

        # print(acc_percent, f1_percent, mcc_percent)
        # sys.exit(0)

        for i in range(1, len(data)-1):
            # print(i)
            pred, upd = scargc.scargc_1NN(data, label_, data[i], label[i], data[i+1], label[i+1], pool_size, n_components, n_features, k)
            pred = np.array(pred, dtype=int)
            # print(label[i])
            pred = pred.flatten()

            # print(pred)
            acc, f1, mcc = metrics.makeBatches(label[i], pred, len(label[i]), step)

            predicted = np.vstack([predicted, pred])
            # print(predicted)
            # sys.exit(0)

            updt += upd
            acc_percent = np.vstack([acc_percent, acc])
            f1_percent = np.vstack([f1_percent, f1])
            mcc_percent = np.vstack([mcc_percent, mcc])


        # print(acc_percent)

        predicted = predicted.flatten()
        lab = label_[tam:]


        score = accuracy_score(lab, predicted)
        f1 = f1_score(lab, predicted, average = 'macro')
        mcc = matthews_corrcoef(lab, predicted)
        # score, f1, mcc = metrics.metrics(acc_percent, lab, predicted, step, f1_type = 'macro')
        # print(a,score, f1, mcc)
    return score, f1, mcc


In [3]:
poolsize = 300
clusters = 2
n_components = 2
# episilon = 0.15
step = 10

In [4]:
base = '/home/god/Documentos/tcc/datasets/'
list = ['1CSurr.txt', '2CDT.txt', '2CHT.txt']# 'NOAA.txt', 'elec.txt', 'keystroke.txt']
# list = ['keystroke.txt']
# list = ['NOAA.txt']#, 'elec.txt']
database = {}

for i in range(0, len(list)):
    database[i] = base + list[i]

# array_ep = [0.05, 0.10, 0.15]
# array_ep = [0.1]
array_k = [50, 25, 12, 7, 5, 3, 1]
array_p = [10, 20, 30]
# array_p = [30]

result = [0.0, 0.0, 0.0]
aux = []
result = np.array(result, dtype=float)

dic = {}

In [5]:
dic = {}
for key, value in database.items():
    dic[key] = {}
    for k in range(0, len(array_k)):
        for p in range(0, len(array_p)):
            adr = value
            dataset, data_labeled, dataset_train, l_train, stream, l_stream, n_features = u.criar_datasets(5, adr)

            score, f1, mcc = doTrain(dataset_train, l_train, step, n_features, array_p[p], array_k[k], n_components)

            aux = np.hstack([score, f1, mcc])
            result = np.vstack([result, aux])

            a = str(array_k[k])
            b = str(array_p[p])
            name = a +' / '+ b
            dic[key][name] = aux

/home/god/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [6]:
for p_id, p_info in dic.items():
    print('dataset :', p_id)

    for key in p_info:
        print(key, ':', p_info[key])

dataset : 0
50 / 10 : [0. 0. 0.]
50 / 20 : [0.63365539 0.58454973 0.17060271]
50 / 30 : [0.80877617 0.78441562 0.57007526]
25 / 10 : [0.63446055 0.57244993 0.15180786]
25 / 20 : [0.84299517 0.82911248 0.65950833]
25 / 30 : [0.83293076 0.80975413 0.62249885]
12 / 10 : [0.81964573 0.80790919 0.62316394]
12 / 20 : [0.83132045 0.81952655 0.64470026]
12 / 30 : [0.81239936 0.79391104 0.58805992]
7 / 10 : [0.84983897 0.83708557 0.67603828]
7 / 20 : [0.84863124 0.83613338 0.6745845 ]
7 / 30 : [0.81723027 0.79813409 0.5962879 ]
5 / 10 : [0.84983897 0.8373231  0.67681101]
5 / 20 : [0.8389694  0.82728831 0.6594208 ]
5 / 30 : [0.80676329 0.78935988 0.57972067]
3 / 10 : [0.85426731 0.8421587  0.68654779]
3 / 20 : [0.84098229 0.83038111 0.66768806]
3 / 30 : [0.80515298 0.78956816 0.58183837]
1 / 10 : [0.83293076 0.8214842  0.64912444]
1 / 20 : [0.81763285 0.80805055 0.62972818]
1 / 30 : [0.78059581 0.76724102 0.54315241]
dataset : 1
50 / 10 : [0. 0. 0.]
50 / 20 : [0. 0. 0.]
50 / 30 : [0. 0. 0.]
25 /